<center><h1> Directed graphical models (Bayesian Networks) </h1></center>

# 1. Basic idea
The directed graphical models are intented to represent the joint distribution $p(\vec{x}|\theta)$,especially for high dimension data.By the **chain rule** of probability,we can always represent a joint distribution as follows
$$
p(x_{1:V})=p(x_1)p(x_2|x_1)p(x_3|x_1,x_2) \dots p(x_V|x_{V-1},\ldots,x_1)
$$
The problem is that when the $V$ is large, the distribution $p(x_V|x_{V-1},\ldots,x_1)$ will be very complicate. The reason is that we have not make any assumptions on the dependence relationship between the data.

The key to efficiently representing large joint distributions is to make some assumptions about the **conditional independence (CI)**. This is the magic of the directed graphical models.
$$
X \perp Y\,|\,Z \longleftrightarrow p(X,Y\,|\,Z)=p(X\,|\,Z)p(Y\,|\,Z) \longleftrightarrow p(X\,|\,Y,Z)=p(X\,|\,Z)
$$

A graphical model(GM) is a way to represent a joint distribution by making **CI** assumptions. The nodes in the graph represent random variables, and the (lack of)edges represent **CI** assumptions. A **directed graphical model(DGM)** is a GM whose graph is a **directed acyclic graph(DAG)**. These are more commonly known as **Bayesian networks**.

The key property of **DAG** is that the nodes can be ordered such that parents come before children using **topological ordering**.Given such an order, we define the **ordered Markov property** to be the assumption that  a node only depends on its immediate parents, not on all predecessors in the ordering

$$
x_s \perp x_{pred(s)\backslash pa(s)}  | x_{pa(s)}
$$

where $pa(s)$ are the parents of node s, and $pred(s)$ are the predecessors of node $s$ in the topological ordering. The joint distribution defined by a graph is given by the product, over all of the nodes of the graph, of a conditional distribution for each node conditioned on the variables corresponding to the parents of that node in the graph. Thus, for a graph with $K$ nodes, the joint distribution is given by
$$
p(x_{1:K})=\prod_{k=1}^K p(x_k|pa_k)
$$
<img src="imgs/1.png" alt="drawing" width="250"/>
The DAG above encodes the following joint distribution
\begin{align}
p(x_1,x_2,x_3,x_4,x_5) &=p(x_1)p(x_2|x_1)p(x_3|x_1)p(x_4|x_2,x_3)p(x_5|x_3) \\
\end{align}

# 2. Some examples
### Polynomial regression
Consider the bayesian polynomial regression model with training dataset $D=\left\{(x_i,y_i)\right\}_{i=1}^N$.
\begin{align}
\vec{t} &=\left(1,x,x^2,\ldots,x^K\right) \\
y&=\mathcal{N}(\vec{t}^T\vec{w},\sigma^2) \\
\vec{w} &\sim \mathcal{N}(\mathbf{0},\frac{1}{\alpha} \mathbf{I})
\end{align}
The random variables in this model are the vector of polynomial coefficients $\vec{w}$ and the observed data $Y=\left(y_1,y_2,\ldots,y_N\right)$. The noise variance $\sigma^2$, the hyperparameter $\alpha$ representing the precision of the Gaussian prior over $\vec{w}$  and the input data $X=\left(x_1,x_2,\ldots x_N\right)$ are **parameters** of the model rather than random variables. In a graphical model, we will denote **observed variables** by shading the corresponding nodes.The other non-observed variable is called **latent variable**.
<img src="imgs/2.png" alt="drawing" width="250"/>
Having observed the values $\left\{y_1,y_2,\ldots,y_N\right\}$ we can, if desired, evaluate the posterior   distribution of the polynomial coefficients $\vec{w}$ using bayes law as follows
$$
p(\vec{w}|Y) \propto p(w) \prod_{n=1}^N p(y_n|\vec{w})
$$
In general, model parameters such as $\vec{w}$ are of little direct interest in themselves,because our ultimate goal is to make predictions for new input values. For predictions, the graphical model that describles this problem is shown below.
<img src="imgs/3.png" alt="drawing" width="250"/>
We can make predictions as follows
\begin{align}
p(y_{new}|Y) &=\int p(y_{new},\vec{w}|Y)d\vec{w} \\
             &=\int p(y_{new}|\vec{w},Y)p(\vec{w}|Y)d\vec{w} \\
             &=\int p(y_{new}|\vec{w})p(\vec{w}|Y)d\vec{w} \\
\end{align}
**The secret is that condition on the observed variables and margin the irrelevant latent variables .**

We can also represent $\sigma^2$ and $\alpha$ as random variable, which lead to more complex models.
\begin{align}
\sigma^2 &\sim Gamma(\alpha_0,\beta_0) \\
\alpha &\sim Gamma(\alpha_1,\beta_1) \\
\end{align}
<img src="imgs/4.png" alt="drawing" width="250"/>
We can get even more complex models by making $x$ being random variable as follows.
<img src="imgs/5.png" alt="drawing" width="250"/>
Therefore, there are no distinct difference between parameters and random variables in DGMs. More random variables, more complex the model.
### Directed Gaussian graphical models
Consider a DGM where all the variable are real-valued and all the conditional distribution have the following form
$$
p(x_t|\vec{x_{pa(t)}})=\mathcal{N}(x_t|\mu_t+\vec{w}_t^T\vec{x_{pa(t)}},\sigma_t^2)
$$
This is called a linear Gaussian distribution.Multiplying all these conditional distribution together results in a large joint Gaussian distribution. Several widely used techniques are examples of linear-Gaussian models, such as probabilistic principal component analysis, factor analysis and linear dynamic systems.
# 3. D-separation
We now give a general statement of the CI property for directed graphs.Consider a general directed graph in which A, B, and C are arbitrary sets of nodes.We wish to ascertain whether a particular conditional
independence statement $ A \perp B\,|\,C$ is implied by a given directed acyclic graph.To do so, we consider all possible paths from any node in $A$ to any node in $B$. Any such path is said to be blocked if it includes a node such that either
* the arrows on the path meet either head-to-tail or tail-to-tail at the node, and the node is in the set $C$.
* the arrows meet head-to-head at the node, and neither the node, nor any of its descendants, is in the set $C$.

If all paths are blocked, then $A$ is said to be d-separated from $B$ by $C$. For example, the graph below, the path from $a$ to $b$ is $a \rightarrow e \rightarrow f \rightarrow b$. The path is blocked by $e$ because it's a head-head node and  neither the node, nor any of its descendants are observed.  The path is blocked by $f$ because it's a tail-tail node and the node is observed. So $a$ and $b$ is independent condition on the observed variables.
<img src="imgs/6.png" alt="drawing" width="250"/>
Another example, the path from $a$ to $b$ is $a \rightarrow e \rightarrow f \rightarrow b$. The path is not blocked by  $f$ because it's a tail-tail node and not observed. The path is not blocked by $e$ because it's a head-head node and its child is observed. So $a$ and $b$ is not independent condition on the observed variables.  
<img src="imgs/7.png" alt="drawing" width="250"/>

# 4. Inference
Suppose we have a set of correlated random variables with joint distribution $p(x_{1:V|\theta})$ (In this section,we are assuming the parameters $\theta$ of the model are known). Let us partition this vector into the **visible variables** $x_v$, which are observed, and the **hidden variables** $x_h$, which are unobserved. Inference refers to computing the posterior distribution of the unknowns given the knowns
\begin{align}
p(x_h|x_v,\theta) &=\frac{p(x_v,x_h|\theta)}{p(x_v|\theta)} \\
                  &=\frac{p(x_v,x_h|\theta)}{\sum_{x_h'}p(x_v,x_h',\theta)} \\
\end{align}

In the hidden variables, only some of them are of interest to us, which denoted as $x_q$. We can compute what we are interested in by **marginalizing out** the rest **nuisance variables** $x_n$.
$$
p(x_q|x_v,\theta)=\sum_{x_n} p(x_h|x_v,\theta)
$$

# 5. Learning
In inference, we assume that the parameter $\theta$ is known to us. But actually, we need to computing a MAP estimate of the parameters given training data.
$$
\hat{\theta}=\underset{\theta}{argmax} \sum_{i=1}^N log\,p(x_{i,v}|\theta)+log\, p(\theta)
$$
where $x_{i,v}$ are visible variables in train data sample i. If we have a uniform prior, $p(\theta) \propto 1$, this reduces to the MLE.

### Learning form complete data
If all the variables are fully observed in each case, so there is no missing data and there are no hidden variables, we say the data is **complete**. For a DGM with complete data, the likelihood is given by
\begin{align}
p(D|\theta) &=\prod_{i=1}^N p(x_i|\theta) \\
            &=\prod_{i=1}^N \prod_{t=1}^V p(x_{it}|x_{i,pa(t)},\theta_t) \\
            &=\prod_{t=1}^V p(D_t|\theta_t)
\end{align}
where $D_t$ is the data associated with the node $t$ and its parents. This is a product of terms, which means likelihood **decomposes** according to the graph structure.

Now suppose that the prior factorizes as well
$$
p(\theta)=\prod_{t=1}^V p(\theta_t)
$$
Then clearly the posterior also factorizes
\begin{align}
p(\theta|D) &\propto p(\theta)p(D|\theta) \\
            &=\prod_{t=1}^V p(\theta_t)p(D_t|\theta_t) 
\end{align}
This means we can compute the posterior of each conditional distribution independently, just as the Naive Bayes classification case.

### Learning with missing and/or latent variables
If we have missing data and/or hidden variables, the likelihood no longer factorizes, and the troubles come. We will discuss in the following chapters.